## 練習問題９

### 9.2.3

**練習9-1**  
決定木による分類

**練習9-2**  
アウトプットはお客さんが投資信託商品を購入してくれるか否か。  
分岐条件と閾値を調べることで、何が足りなかったのか分析できる。  

**練習9-3**  

In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
df = pd.read_csv('Bank.csv')
df.head()

,id,age,job,marital,education,default,amount,housing,loan,contact,day,month,duration,campaign,previous,y
0,1,39,blue-collar,married,secondary,no,1756.0,yes,no,cellular,3,apr,370.055237,1,0,1
1,2,51,entrepreneur,married,primary,no,1443.0,no,no,cellular,18,feb,233.998933,10,0,1
2,3,36,management,single,tertiary,no,436.0,no,no,cellular,13,apr,NaN,1,2,0
3,4,63,retired,married,secondary,no,474.0,no,no,cellular,25,jan,252.525808,1,0,0
4,5,31,management,single,tertiary,no,354.0,no,no,cellular,30,apr,NaN,1,2,0


In [117]:
# ダミー変数化
col = ['job','marital','education','default','housing','loan','contact','month']
str_df = df[col]
str_dummies_df = pd.get_dummies(str_df,drop_first = True, dtype = int)
str_dummies_df.head()

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
num_df = df.drop(col,axis = 1)

In [119]:
df2 = pd.concat([str_dummies_df,num_df,str_df], axis=1)
df2.shape

(27128, 48)

In [120]:
df2.columns

Index(['job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'default_yes', 'housing_yes',
       'loan_yes', 'contact_sending _document', 'contact_telephone',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'id', 'age', 'amount', 'day', 'duration', 'campaign',
       'previous', 'y', 'job', 'marital', 'education', 'default', 'housing',
       'loan', 'contact', 'month'],
      dtype='object')

In [121]:
# データの分割
train_val,test = train_test_split(df2, test_size = 0.2, random_state = 0)

In [122]:
# 欠損値処理
train_val.isnull().sum()

job_blue-collar                 0
job_entrepreneur                0
job_housemaid                   0
job_management                  0
job_retired                     0
job_self-employed               0
job_services                    0
job_student                     0
job_technician                  0
job_unemployed                  0
job_unknown                     0
marital_married                 0
marital_single                  0
education_secondary             0
education_tertiary              0
education_unknown               0
default_yes                     0
housing_yes                     0
loan_yes                        0
contact_sending _document       0
contact_telephone               0
month_aug                       0
month_dec                       0
month_feb                       0
month_jan                       0
month_jul                       0
month_jun                       0
month_mar                       0
month_may                       0
month_nov     

In [123]:
# 平均値で穴埋め
train_val_mean = train_val['duration'].mean()
train_val2 = train_val.fillna(train_val_mean)
train_val2['duration'].isnull().sum()

0

In [124]:
# 不均衡データの確認
train_val2['y'].value_counts()

y
0    14772
1     6930
Name: count, dtype: int64

In [125]:
# 訓練データと検証データの分割
t = train_val2['y']
x_temp = train_val2.drop(col, axis = 1)
x = x_temp.drop(['id', 'y', 'day'], axis = 1)
x_train,x_val,y_train,y_val = train_test_split(x, t, test_size = 0.2, random_state = 0)

In [126]:
# モデルの作成
model = tree.DecisionTreeClassifier(max_depth = 5, random_state = 0, class_weight = 'balanced')
model.fit(x_train,y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,'balanced'


In [127]:
# モデルの評価
model.score(x_val,y_val)

0.7608845888044229

### 練習9-4

In [155]:
def learn(x,t,d):
    x_train,x_val,y_train,y_val = train_test_split(x,t,test_size = 0.2, random_state = 0)
    model = tree.DecisionTreeClassifier(max_depth = d, random_state = 0, class_weight = 'balanced')
    model.fit(x_train,y_train)
    score_train = model.score(x_train,y_train)
    score_val = model.score(x_val, y_val)
    datas =  [x_train,x_val,y_train,y_val]
    return score_train, score_val, model, datas

In [158]:
# 木の深さを変えてみる
for d in range(1,15):
    s1,s2, model, datas = learn(x, t, d)
    print(d,s1,s2)

1 0.750878405621796 0.7472932504031329
2 0.750878405621796 0.7472932504031329
3 0.7557168365877541 0.7521308454273209
4 0.7501296008294454 0.7509790370882286
5 0.7740337538160245 0.770099055517162
6 0.7892978515062496 0.7878369039391845
7 0.8356661482633488 0.8233126007832297
8 0.8504118426357928 0.8272287491361437
9 0.8609527100973446 0.8184750057590416
10 0.8767928114739935 0.816171389080857
11 0.8891192903634584 0.8154803040774016
12 0.905247393583319 0.8136374107348537
13 0.9221243015955302 0.8136374107348537
14 0.9391164103450262 0.8101819857175766


In [159]:
# 木の深さ10でテストデータで評価
model = tree.DecisionTreeClassifier(max_depth = 10, random_state = 0, class_weight = 'balanced')
model.fit(x,t)
test2 = test.copy()
test2 = test2.fillna(train_val_mean)
test_y = test2['y']
test_x = test2.drop(col, axis = 1)
test_x = test_x.drop(['y', 'day', 'id'], axis = 1)
model.score(test_x, test_y)

0.680796166605234

In [160]:
# 欠損値を中央値で埋めたバージョン
test3 = test.copy()
train_val_median = train_val['duration'].median()
test3 = test3.fillna(train_val_median)
test_y = test3['y']
test_x = test3.drop(col, axis = 1)
test_x = test_x.drop(['y', 'day', 'id'], axis = 1)
model.score(test_x, test_y)

0.6791374861776631

In [161]:
# yと関係がありそうなものを探す。
for name in str_df.columns:
    print(train_val.groupby(name)['y'].mean())
    print("next=========")

job
admin.           0.311699
blue-collar      0.354396
entrepreneur     0.382749
housemaid        0.282430
management       0.308227
retired          0.368468
self-employed    0.292047
services         0.328203
student          0.153846
technician       0.305648
unemployed       0.238019
unknown          0.308725
Name: y, dtype: float64
next=========
marital
divorced    0.331283
married     0.340136
single      0.269984
Name: y, dtype: float64
next=========
education
primary      0.350904
secondary    0.319463
tertiary     0.299780
unknown      0.339188
Name: y, dtype: float64
next=========
default
no     0.320747
yes    0.240209
Name: y, dtype: float64
next=========
housing
no     0.215788
yes    0.401488
Name: y, dtype: float64
next=========
loan
no     0.299588
yes    0.422426
Name: y, dtype: float64
next=========
contact
cellular             0.276659
sending _document    0.415632
telephone            0.315481
Name: y, dtype: float64
next=========
month
apr    0.316501
aug    0.247

これよりdefault,housing,loan,contactが関係してそう。

In [162]:
print(pd.pivot_table(train_val, index = 'default', columns = 'housing', values = 'duration'))
print(pd.pivot_table(train_val, index = 'default', columns = 'loan', values = 'duration'))
print(pd.pivot_table(train_val, index = 'default', columns = 'contact', values = 'duration'))
print(pd.pivot_table(train_val, index = 'housing', columns = 'loan', values = 'duration'))
print(pd.pivot_table(train_val, index = 'housing', columns = 'contact', values = 'duration'))
print(pd.pivot_table(train_val, index = 'loan', columns = 'contact', values = 'duration'))

housing          no         yes
default                        
no       251.954397  349.927993
yes      265.495061  354.989629
loan             no         yes
default                        
no       304.087711  383.670117
yes      297.949921  357.848790
contact    cellular  sending _document   telephone
default                                           
no       313.134144         328.645498  318.479162
yes      317.385927         329.936246  330.663915
loan             no         yes
housing                        
no       237.824421  299.867693
yes      332.885094  439.809380
contact    cellular  sending _document   telephone
housing                                           
no       250.354476         248.318302  272.307548
yes      348.914190         349.254800  371.123510
contact    cellular  sending _document   telephone
loan                                              
no       298.068531         314.537311  307.323846
yes      374.565688         402.179225  380.765335


結果を見るとhousingとloanが強く関係してそうに見える。よってhousingとloanのピボットテーブルを使って、durationの欠損値を埋める。

In [163]:
print(pd.pivot_table(train_val, index = 'housing', columns = 'loan', values = 'duration'))

loan             no         yes
housing                        
no       237.824421  299.867693
yes      332.885094  439.809380


In [164]:
def nan_fill(train_val):
    isnull = train_val['duration'].isnull()

    #train_valを変えたくないので、py基本文法のcopyメソッドでコピーを作って
    #train_val2に代入
    train_val2 = train_val.copy()
    #housing=yesの処理
    train_val2.loc[(isnull) & (train_val2['housing']=='yes') 
                  &(train_val2['loan']=='yes'),'duration' ]=440
    train_val2.loc[(isnull) & (train_val2['housing']=='yes') 
                  &(train_val2['loan']=='no'),'duration' ]=333

    #housing=noの処理
    train_val2.loc[(isnull) & (train_val2['housing']=='no') 
                  &(train_val2['loan']=='yes'),'duration' ]=300
    train_val2.loc[(isnull) & (train_val2['housing']=='no') 
                  &(train_val2['loan']=='no'),'duration' ]=238
    
    return train_val2
train_val2=nan_fill(train_val)

In [165]:
train_val2.groupby('y')['duration'].median()

y
0    263.248364
1    348.740807
Name: duration, dtype: float64

In [166]:
train_val2.groupby('y')['amount'].median()

y
0    452.5
1    426.5
Name: amount, dtype: float64

In [167]:
train_val2.groupby('y')['campaign'].median()

y
0    2.0
1    3.0
Name: campaign, dtype: float64

In [168]:
train_val2.groupby('y')['age'].median()

y
0    38.0
1    41.0
Name: age, dtype: float64

上記の結果より、durationが大きいほどyが１になりやすそう。

In [177]:
#ここでいったん、learn関数で分析

t =train_val2['y']

x = train_val2.drop(col,axis=1)
x =x.drop(['id','y','day'],axis=1)
#x =train_val2[xcol]
for i in range(1,15):
    s1,s2,model,datas = learn(x,t,i)
    print(i,s1,s2)

1 0.750878405621796 0.7472932504031329
2 0.750878405621796 0.7472932504031329
3 0.7557168365877541 0.7521308454273209
4 0.7501296008294454 0.7509790370882286
5 0.7740337538160245 0.770099055517162
6 0.7892978515062496 0.7878369039391845
7 0.8356661482633488 0.8233126007832297
8 0.8504118426357928 0.8272287491361437
9 0.8609527100973446 0.8184750057590416
10 0.8767928114739935 0.816171389080857
11 0.8891192903634584 0.8154803040774016
12 0.905247393583319 0.8136374107348537
13 0.9221243015955302 0.8136374107348537
14 0.9391164103450262 0.8101819857175766


In [185]:
#どのような間違い方をしているのか確認
s1,s2,model,datas = learn(x,t,10)

#訓練データでの予測結果と実際の値の2軸で個数集計flagがFalseならば、検証データで集計
def syuukei(model,datas,flag=False):
    if flag:
        pre=model.predict(datas[0])
        y_val=datas[2]
    else:
        pre=model.predict(datas[1])
        y_val=datas[3]
    data={
        "pred":pre,
        "true":y_val
    }
    tmp=pd.DataFrame(data)
    return tmp,pd.pivot_table(tmp,index="true",columns="pred",values="true",aggfunc=len)
tmp,a=syuukei(model,datas,False)
a

pred,0,1
true,,
0,2416,547
1,251,1127


In [184]:
# 数値列のみを明示的に抽出
num_cols = true_df.select_dtypes(include="number").columns

# y=0 で正しく予測された人 / 誤って予測された人の平均を計算
true_df_mean  = true_df[num_cols].mean()
false_df_mean = false_df[num_cols].mean()

# 比較して可視化
pd.concat(
    [true_df_mean.loc["age":], false_df_mean.loc["age":]],
    axis=1,
    keys=["True Negative", "False Positive"]
).plot(kind="bar")


NameError: name 'true_df' is not defined

In [175]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
tmp2=train_val2.drop(col,axis=1)
sc_data = sc.fit_transform(tmp2)
sc_df = pd.DataFrame(sc_data,columns=tmp2.columns,index=tmp2.index)

true_df=sc_df.loc[true]
false_df=sc_df.loc[false]
true_df
temp2=pd.concat([false_df.mean()["age":],true_df.mean()["age":]],axis=1)
temp2.plot(kind="bar")

NameError: name 'true' is not defined

In [173]:
model_tree=tree.DecisionTreeClassifier(max_depth=10,random_state=10,class_weight="balanced")
model_tree.fit(x,t)

test2=nan_fill(test)
t =test2['y']
x = test2.drop(col,axis=1)
x =x.drop(['id','y','day'],axis=1)
model_tree.score(x,t)

ValueError: Number of labels=5426 does not match number of samples=21702